<a href="https://colab.research.google.com/github/changnos/2021-2-OSSP1-NotScary-2/blob/main/trimmer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**장구 소리 전처리**
1.   파일 불러오기
2.   소리 나누기

- 장구 소리 데이터 드라이브에 업로드
- 장구별 "덩, 덕, 쿵" 각 10회 이상 녹음
- 장구별 녹음 데이터에서 "덩, 덕, 쿵"(덩 10개, 덕 10개, 쿵 10개) 소리 데이터 추출


##**1. 파일 불러오기**


In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.io import wavfile
from glob import glob
from tqdm import tqdm

%matplotlib inline

sns.set_style('darkgrid')

In [ ]:
!sudo pip install numba
!sudo pip install llvmlite
!sudo pip install librosa

#한글 깨짐 방지
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/janggu_data'
file_list = os.listdir(path + '/original')

for file in file_list:
    print(file)

##**2. 소리 나누기**

In [ ]:
import librosa.display, librosa
import soundfile as sf

def trimData(file):
    y, sr = librosa.load(path + '/original/' + file)
    D = np.abs(librosa.stft(y))
    onset_env = librosa.onset.onset_strength(y=y, sr=sr,
                                            hop_length=512,
                                            aggregate=np.median)
    peaks = librosa.util.peak_pick(onset_env, pre_max=1, post_max=3, pre_avg=3, post_avg=5, delta=5, wait=10)
    D = np.abs(librosa.stft(y))
    D_db = librosa.amplitude_to_db(D, ref=np.max)
    times = librosa.times_like(onset_env, sr=sr, hop_length=512)
    tmp2 = np.array([len(times)-1])
    peaks = np.append(peaks, tmp2)

    for i in range(len(peaks) - 1):
        if(i == 0 or i > 10): continue
        start_frame = peaks[i]-5
        end_frame = peaks[i + 1]-5
        term = end_frame - start_frame
        segment = D_db[:, start_frame:end_frame]
        ny = y[round(start_frame*512):round(end_frame*512)]
        sf.write(path + '/trimmed/' + file[0:-4] + '_' + str(i-1) + '.wav', ny, sr)

In [ ]:
for file in file_list:
    trimData(file);

In [ ]:
import shutil

src = '/content/drive/MyDrive/janggu_data/original/'
dest = '/content/drive/MyDrive/janggu_data/used/'

files = os.listdir(src)

for file in file_list:
    shutil.move(src + file, dest)